In [ ]:
# Live Differential Finder

import requests
import pandas as pd

# URLs for data
PLAYER_API_URL = "https://fantasy.premierleague.com/api/bootstrap-static/"
FIXTURE_API_URL = "https://fantasy.premierleague.com/api/fixtures/"

# Fetch data
player_response = requests.get(PLAYER_API_URL)
fixture_response = requests.get(FIXTURE_API_URL)

if player_response.status_code == 200 and fixture_response.status_code == 200:
    # Parse JSON data
    data = player_response.json()
    fixtures = fixture_response.json()
    
    players = data["elements"]  
    teams = {team["id"]: team["name"] for team in data["teams"]}  # Team mapping
    positions = {1: "Goalkeeper", 2: "Defender", 3: "Midfielder", 4: "Forward", 5: "Manager"}  # Position mapping

    # Exclude Managers (Assuming they have element_type = 5)
    players = [player for player in players if player["element_type"] != 5]

    # Extract fixture difficulty for each team
    fixture_difficulty = {}
    for fixture in fixtures:
        home_team = fixture["team_h"]
        away_team = fixture["team_a"]
        fixture_difficulty.setdefault(home_team, []).append(fixture["team_h_difficulty"])
        fixture_difficulty.setdefault(away_team, []).append(fixture["team_a_difficulty"])

    # Compute average fixture difficulty for each team (next 3 matches)
    avg_fixture_difficulty = {
        team_id: round(sum(difficulties[:3]) / len(difficulties[:3]), 2) 
        if difficulties else None
        for team_id, difficulties in fixture_difficulty.items()
    }

    player_data = []
    for player in players:
        team_id = player["team"]
        position_id = player["element_type"]

        # Extracting additional features
        price = player["now_cost"] / 10
        total_points = player["total_points"]
        form = float(player["form"])
        xG = float(player.get("expected_goals", 0))  # Handle missing values
        xA = float(player.get("expected_assists", 0))
        xGI = xG + xA
        ppm = round(total_points / price, 2) if price > 0 else 0  # Points per million
        minutes_per_game = round(player["minutes"] / (player["total_points"] / form) if form > 0 else 1, 2)

        # Custom differential score (higher form, lower ownership, good xGI)
        diff_score = round((form * 2 + xGI * 1.5) / (float(player["selected_by_percent"]) + 1), 2)

        player_data.append({
            "Name": player["web_name"],
            "Team": teams.get(team_id, "Unknown"),
            "Position": positions.get(position_id, "Unknown"),
            "Price": price,
            "Ownership %": float(player["selected_by_percent"]),
            "Expected Goals": xG,
            "Expected Assists": xA,
            "Expected Goal Involvement (xGI)": xGI,
            "Total Points": total_points,
            "Form": form,
            "Points per Million (PPM)": ppm,
            "Minutes per Game": minutes_per_game,
            "Fixture Difficulty (Next 3)": avg_fixture_difficulty.get(team_id, None),
            "Differential Score": diff_score  # New custom metric
        })

    df = pd.DataFrame(player_data)

    # Find differentials (Low ownership < 10% and good form > 2)
    df_differentials = df[(df["Ownership %"] < 10) & (df["Form"] > 2)]
    df_differentials = df_differentials.sort_values(by="Differential Score", ascending=False)

    # Save full data & differentials
    df.to_csv("fpl_enhanced_data.csv", index=False)
    df_differentials.to_csv("fpl_differentials.csv", index=False)

    print("Top Differentials:\n", df_differentials.head(10))

else:
    print("Failed to fetch FPL data. Check your internet connection!")


Top Differentials:
            Name         Team    Position  Price  Ownership %  Expected Goals  \
98    Tavernier  Bournemouth  Midfielder    5.4          0.2            4.48   
80     Christie  Bournemouth  Midfielder    4.9          0.2            1.77   
601       Aribo  Southampton  Midfielder    4.9          0.1            2.54   
718  Bellegarde       Wolves  Midfielder    4.9          0.1            0.82   
681       Danso        Spurs    Defender    4.5          0.0            0.04   
136      Schade    Brentford  Midfielder    5.1          1.0            5.20   
172      Minteh     Brighton  Midfielder    5.0          0.8            2.79   
25       Merino      Arsenal  Midfielder    6.0          0.4            2.98   
662      Spence        Spurs    Defender    4.4          0.3            0.17   
79       Brooks  Bournemouth  Midfielder    4.9          0.0            1.66   

     Expected Assists  Expected Goal Involvement (xGI)  Total Points  Form  \
98               2.18

In [ ]:
# Predict Future Points

import requests
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Fetch FPL data
PLAYER_API_URL = "https://fantasy.premierleague.com/api/bootstrap-static/"
response = requests.get(PLAYER_API_URL)

if response.status_code == 200:
    data = response.json()
    players = data["elements"]
    
    # Team mapping
    teams = {team["id"]: team["name"] for team in data["teams"]}  
    positions = {1: "Goalkeeper", 2: "Defender", 3: "Midfielder", 4: "Forward", 5: "Manager"}

    # Exclude Managers
    players = [player for player in players if player["element_type"] != 5]

    player_data = []
    for player in players:
        team_id = player["team"]
        position_id = player["element_type"]

        # Feature extraction
        price = player["now_cost"] / 10
        total_points = player["total_points"]
        form = float(player["form"])
        xG = float(player.get("expected_goals", 0))
        xA = float(player.get("expected_assists", 0))
        xGI = xG + xA
        ppm = round(total_points / price, 2) if price > 0 else 0
        minutes_per_game = round(player["minutes"] / (player["total_points"] / form) if form > 0 else 1, 2)
        future_points = player.get("event_points", 0)  # Target variable (change this when historical data is available)

        player_data.append({
            "Name": player["web_name"],
            "Team": teams.get(team_id, "Unknown"),
            "Position": positions.get(position_id, "Unknown"),
            "Price": price,
            "Ownership %": float(player["selected_by_percent"]),
            "Expected Goals": xG,
            "Expected Assists": xA,
            "Expected Goal Involvement (xGI)": xGI,
            "Total Points": total_points,
            "Form": form,
            "Points per Million (PPM)": ppm,
            "Minutes per Game": minutes_per_game,
            "Future Points": future_points  # Target variable
        })

    df = pd.DataFrame(player_data)

    # Select features & target
    features = ["Form", "Expected Goals", "Expected Assists", "Total Points", "Ownership %", "Price"]
    target = "Future Points"

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.2, random_state=42)

    # Train Random Forest model
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Evaluate model
    mae = mean_absolute_error(y_test, y_pred)
    print(f"Mean Absolute Error: {mae:.2f}")

    # Add predicted points to DataFrame
    df["Predicted Future Points"] = model.predict(df[features])

    # Sort by highest predicted points
    top_predictions = df.sort_values(by="Predicted Future Points", ascending=False)

    # Save results
    top_predictions.to_csv("fpl_predictions.csv", index=False)
    print("Top Players for Next Gameweeks:\n", top_predictions.head(10))

else:
    print("Failed to fetch FPL data!")


Mean Absolute Error: 0.66
Top Players for Next Gameweeks:
                  Name           Team    Position  Price  Ownership %  \
435           M.Salah      Liverpool  Midfielder   13.7         71.9   
487          Marmoush       Man City     Forward    7.1          6.1   
172            Minteh       Brighton  Midfielder    5.0          0.8   
419  Alexander-Arnold      Liverpool    Defender    7.4         32.6   
434         Luis Díaz      Liverpool  Midfielder    7.5         17.5   
25             Merino        Arsenal  Midfielder    6.0          0.4   
80           Christie    Bournemouth  Midfielder    4.9          0.2   
459        Ederson M.       Man City  Goalkeeper    5.3          4.7   
302           Alcaraz        Everton  Midfielder    4.9          0.1   
587              Wood  Nott'm Forest     Forward    7.1         44.7   

     Expected Goals  Expected Assists  Expected Goal Involvement (xGI)  \
435           19.13              6.02                            25.15   
